In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import NMF
from openTSNE import TSNEEmbedding, affinity, initialization
from tqdm import tqdm
from collections import deque
import harmonypy as hm
import networkx as nx
from threadpoolctl import threadpool_limits, threadpool_info

# configure thread limits
threadpool_limits(limits=8)
os.environ['OMP_NUM_THREADS'] = '6'

In [ ]:
data = pd.read_hdf("20241103_featsel_lba.h5", key="table")

coordinates = data[['Section', 'xccf', 'yccf', 'zccf']]
coordinates['Section'] = coordinates['Section'].astype(int)

metdat = data.iloc[:,:23]
data = data.iloc[:,23:]
data

## Define a seeding procedure to initialize NMF by automatically detecting a reasonable number of factors, as high as possible while preserving good modularity of lipid correlation blocks

In [ ]:
# use NMF to decompose the data into factors

def compute_seeded_NMF(data):  # data is a dataframe pixels x lipids
    # 1. calculate the correlation matrix of this dataset
    corr = np.corrcoef(data.values.T)
    corr_matrix = np.abs(corr)  # anticorrelated lipids convey the same info
    np.fill_diagonal(corr_matrix, 0)
    
    adata = anndata.AnnData(X=np.zeros_like(corr_matrix))
    adata.obsp['connectivities'] = csr_matrix(corr_matrix)
    adata.uns['neighbors'] = {
        'connectivities_key': 'connectivities',
        'distances_key': 'distances',
        'params': {'n_neighbors': 10, 'method': 'custom'}
    }
    
    G = nx.from_numpy_array(corr_matrix)
    
    # span reasonable Leiden resolution parameters
    gamma_values = np.linspace(0.8, 1.5, num=100)
    num_communities = []
    modularity_scores = []
    objective_values = []
    
    for gamma in gamma_values:
        sc.tl.leiden(adata, resolution=gamma, key_added=f'leiden_{gamma}')
        clusters = adata.obs[f'leiden_{gamma}'].astype(int).values
        num_comms = len(np.unique(clusters))
        num_communities.append(num_comms)
        partition = {i: clusters[i] for i in range(len(clusters))}
        modularity = nx.community.modularity(G, [np.where(clusters == i)[0] for i in range(num_comms)])
        modularity_scores.append(modularity)
    
    # 3. pick a number of blocks that is relatively high while preserving good modularity
    epsilon = 1e-10
    alpha = 0.7  # controls the weight of modularity vs pushing higher the number of communities
    for Q, N_c in zip(modularity_scores, num_communities):
        f_gamma = Q**alpha * np.log(N_c + 1 + epsilon)
        objective_values.append(f_gamma)
        
    plt.plot(np.arange(len(objective_values)), objective_values)
    plt.title("obj")
    plt.show()
    
    plt.plot(np.arange(len(modularity_scores)), modularity_scores)
    plt.title("mod")
    plt.show()
        
    plt.plot(np.arange(len(num_communities)), num_communities)
    plt.title("ncomms")
    plt.show()
    
    max_index = np.argmax(objective_values)
    best_gamma = gamma_values[max_index]
    best_modularity = modularity_scores[max_index]
    best_num_comms = num_communities[max_index]
    print(f'Number of communities at best gamma: {best_num_comms}')
    
    sc.tl.leiden(adata, resolution=best_gamma, key_added='leiden_best') # run Leiden one final time with best parameters
    clusters = adata.obs['leiden_best'].astype(int).values
    print(clusters)
    
    N_factors = best_num_comms
    
    # 4. pick a representative lipid from each block, use to initialize W
    dist = 1 - corr_matrix
    np.fill_diagonal(dist, 0)
    dist = np.maximum(dist, dist.T)  # as numerical instability makes it unreasonably asymmetric
    dist_condensed = squareform(dist, checks=True)
    representatives = []
    
    for i in range(0, N_factors):
        cluster_members = np.where(clusters == i)[0]
        print(cluster_members)
        if len(cluster_members) > 0:  # find most central feature in cluster
            mean_dist = dist[cluster_members][:, cluster_members].mean(axis=1)
            central_idx = cluster_members[np.argmin(mean_dist)]
            representatives.append(central_idx)
    
    W_init = data.values[:, representatives]
    
    # 5. initialize H as a subset of the correlation matrix
    H_init = corr[representatives,:]
    H_init[H_init < 0.] = 0.  # only positive correlated can contribute by def in NMF
    
    # 6. compute the NMF with this initialization and rank N
    N_factors = W_init.shape[1]
    nmf = NMF(
        n_components=N_factors,
        init='custom',
        random_state=42
    )
    data_offset = data - np.min(data) + 1e-7
    
    data_offset = np.ascontiguousarray(data_offset)
    W_init = np.ascontiguousarray(W_init)
    H_init = np.ascontiguousarray(H_init)
    
    nmf_result = nmf.fit_transform(
        data_offset,
        W=W_init,
        H=H_init
    )
    nmfdf = pd.DataFrame(nmf_result, index=data.index)
    factor_to_lipid = nmf.components_
    
    return nmfdf, factor_to_lipid, N_factors, nmf

In [ ]:
nmfdf, factor_to_lipid, N_factors, nmf = compute_seeded_NMF(data)

In [ ]:
# save to file

np.save("factor_to_lipid.npy", factor_to_lipid)
nmfdf.to_hdf("nmfdf.h5ad", key="table")

## Check visually the NMF embeddings

In [ ]:
for PC_I in range(0, N_factors):

    results = []
    filtered_data = pd.concat([coordinates.loc[data.index,:], pd.DataFrame(nmf_result[:,PC_I], index=coordinates.loc[data.index,:].index,columns=["test"])], axis=1)

    currentPC = "test"
    
    for section in filtered_data['Section'].unique():
        subset = filtered_data[filtered_data['Section'] == section]

        perc_2 = subset[currentPC].quantile(0.02) 
        perc_98 = subset[currentPC].quantile(0.98)

        results.append([section, perc_2, perc_98])
    percentile_df = pd.DataFrame(results, columns=['Section', '2-perc', '98-perc'])
    med2p = percentile_df['2-perc'].median()
    med98p = percentile_df['98-perc'].median()

    cmap = plt.cm.PuOr

    fig, axes = plt.subplots(4, 8, figsize=(20, 10))
    axes = axes.flatten()

    for section in range(1, 33):
        ax = axes[section - 1]
        ddf = filtered_data[(filtered_data['Section'] == section)]

        ax.scatter(ddf['zccf'], -ddf['yccf'], c=ddf[currentPC], cmap="PuOr", s=0.5,rasterized=True, vmin=med2p, vmax=med98p) 
        ax.axis('off')
        ax.set_aspect('equal')

    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    norm = Normalize(vmin=med2p, vmax=med98p)
    sm = ScalarMappable(norm=norm, cmap=cmap)
    fig.colorbar(sm, cax=cbar_ax)

    plt.tight_layout(rect=[0, 0, 0.9, 1])
    plt.show()

## Use the NMF computed on the atlas to transform the other brains

In [ ]:
# transform the whole dataset with the NMF learnt on the atlas

allbrains = pd.read_hdf("/data/luca/lipidatlas/ManuscriptAnalysisRound3/20241103_pixels_allips_allbrains_allen_pixelcleaned.h5ad")

tmp = allbrains.iloc[:,:-23]
tmp.columns = tmp.columns.astype(float).astype(str)
tmp = tmp.loc[:, data.columns.astype(float).astype(str)]
tmp

nmf_allbrains = nmf.transform(tmp.fillna(0.0001))

# save it to file
nmfall = pd.DataFrame(nmf_allbrains, index=tmp.index)
nmfall.to_hdf("seeded_nmf_allbrains.h5ad", key="table")

In [ ]:
metdat = allbrains.iloc[:, -23:]

# remove the bad sections
nmfall = nmfall.loc[metdat['BadSection'] == 0,:]
nmfall

## Use Harmony to integrate all brains in NMF space with full covariates awareness

In [ ]:
batches = metdat[['SectionID','Sample', 'Sex', 'Condition']].astype("category")
batches = batches.loc[nmfall.index,:]
batches['SectionID'] = batches['SectionID'].astype("category")
batches['Sample'] = batches['Sample'].astype("category")
batches['Sex'] = batches['Sex'].astype("category")
batches['Condition'] = batches['Condition'].astype("category")
batchessub = batches.copy()
unique_values = sorted(batchessub['SectionID'].unique())
value_mapping = {old_value: new_index for new_index, old_value in enumerate(unique_values)}
batchessub['SectionID'] = batchessub['SectionID'].map(value_mapping)
batchessub['SectionID'] = batchessub['SectionID'].astype("category")
batchessub

In [ ]:
nmfall_mat = nmfall
meta_nmfall = batchessub
vars_use = list(batchessub.columns)

ho = hm.run_harmony(nmfall_mat, meta_nmfall, vars_use)

corrected_chunk = pd.DataFrame(
    ho.Z_corr.T,
    index=nmfall_mat.index,
    columns=nmfall_mat.columns
)

corrected_chunk.to_hdf("corrected_nmfall_nochunking.h5ad", key="table")

In [ ]:
corrected_nmfall = corrected_chunk

coordinates = metdat[['x', 'y', 'SectionID']]
coordinates = coordinates.loc[corrected_nmfall.index,:]

## Check batch integration in embeddings with spatial plots

In [ ]:
# note: the harmonized components that use all covariates to remove any sample-specific features and achieve full integration
# which is very important to do label transfer and annotate clusters regardless of their specific sex/condition, to then use the native lipids to do 1-1 differential comparisons

for PC_I in range(N_factors):

    results = []
    filtered_data = pd.concat([coordinates, pd.DataFrame(corrected_nmfall.values[:,PC_I], index=coordinates.index,columns=["test"])], axis=1)
    filtered_data = filtered_data.dropna(subset=['SectionID'])
   
    currentPC = "test"
    
    for section in filtered_data['SectionID'].unique():
        subset = filtered_data[filtered_data['SectionID'] == section]

        perc_2 = subset[currentPC].quantile(0.02)
        perc_98 = subset[currentPC].quantile(0.98)

        results.append([section, perc_2, perc_98])
    percentile_df = pd.DataFrame(results, columns=['SectionID', '2-perc', '98-perc'])
    med2p = percentile_df['2-perc'].median()
    med98p = percentile_df['98-perc'].median()

    cmap = plt.cm.PuOr

    fig, axes = plt.subplots(14, 10, figsize=(20, 38))
    axes = axes.flatten()

    for section in range(1, np.max(filtered_data['SectionID'].astype(int))+1):
        ax = axes[section - 1]
        try:
            ddf = filtered_data[(filtered_data['SectionID'] == section)]

            ax.scatter(ddf['y'], -ddf['x'], c=ddf[currentPC], cmap="PuOr", s=0.5,rasterized=True, vmin=med2p, vmax=med98p) 
            ax.axis('off')
            ax.set_aspect('equal')
        except:
            continue

    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    norm = Normalize(vmin=med2p, vmax=med98p)
    sm = ScalarMappable(norm=norm, cmap=cmap)
    fig.colorbar(sm, cax=cbar_ax)

    plt.tight_layout(rect=[0, 0, 0.9, 1])
    plt.show()

## Check coembedding on a t-SNE

In [ ]:
embds = corrected_nmfall#[::5] # do a minimal downsampling

scaler = StandardScaler()
x_train = scaler.fit_transform(embds)

affinities_train = affinity.PerplexityBasedNN(
    x_train,
    perplexity=30,
    metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
)

init_train = x_train[:,[2, 9]] # initialize with two uncorrelated NMFs, note this affects results a bit

embedding_train = TSNEEmbedding(
    init_train,
    affinities_train,
    negative_gradient_method="fft",
    n_jobs=8,
    verbose=True,
)

embedding_train_1 = embedding_train.optimize(n_iter=500, exaggeration=1.2)

embedding_train_N = embedding_train_1.optimize(n_iter=100, exaggeration=2.5)

In [ ]:
tsne = embedding_train_N

np.save("globaltsne.npy", tsne)

tsne_df = pd.DataFrame(tsne, index = embds.index)
tsne_df.to_hdf("tsne_df.h5ad", key="table")

In [ ]:
metdat = pd.read_hdf("metadata.h5ad", key="table")

In [ ]:
# check if samples / sections / conditions are well integrated vs anatomical regions defining regions

labels = metdat.loc[embds.index, "Sample"]
plt.scatter(tsne[:, 0], tsne[:, 1], c=labels.astype("category").cat.codes, cmap="tab20", s=0.005, alpha=0.3, 
            rasterized=True)
plt.show()

labels = metdat.loc[embds.index, 'Condition']
plt.scatter(tsne[:, 0], tsne[:, 1], c=labels.astype("category").cat.codes, cmap="tab20", s=0.005, alpha=0.3, 
            rasterized=True)
plt.show()

labels = metdat.loc[embds.index, 'Sex']
plt.scatter(tsne[:, 0], tsne[:, 1], c=labels.astype("category").cat.codes, cmap="viridis", s=0.005, alpha=0.3, 
            rasterized=True)
plt.show()

labels = metdat.loc[embds.index, 'allencolor']
plt.scatter(tsne[:, 0][labels!="#000000"], tsne[:, 1][labels!="#000000"], c=labels[labels!="#000000"], s=0.5, alpha=0.3, 
            rasterized=True)
plt.show()